In [2]:
import json
import requests
import pandas as pd
import time
from datetime import date, datetime, timedelta
from collections import defaultdict

In [3]:
def get_data(url, start=None, end=None):
    '''
    Send API request for data between the start and end dates
    '''
    api_key = open("api_key.txt", "r").read()
    
    header = {
        "Authorization": api_key,
        "Accept": "application/vnd.api+json"
    }
    
    if not start and not end:
        r = requests.get(url, headers=header)
    else:
        start = str(start)[:10] + 'T' + str(start)[11:] + 'Z'
        end = str(end)[:10] + 'T' + str(end)[11:] + 'Z'
        query = {
            "sort": "createdAt",
            "filter[createdAt-start]": start,
            "filter[createdAt-end]": end
        }
        r = requests.get(url, headers=header, params=query)
    
    return r.json()

In [4]:
url = "https://api.dc01.gamelockerapp.com/shards/global/matches"

start = datetime(2018, 4, 1, 0, 0, 0, 0)
end = start + timedelta(days=1)

data = get_data(url, start, end)

print(json.dumps(data, indent=4))

{
    "data": [
        {
            "type": "match",
            "id": "34F63A9E2F544B53913C0AD840572330",
            "attributes": {
                "createdAt": "2018-04-01T00:00:00Z",
                "duration": 303,
                "gameMode": "1733162751",
                "patchVersion": "44504",
                "shardId": "global",
                "stats": {
                    "mapID": "319DDC57E70174B6C85EF137BAF34E9E",
                    "type": "QUICK2V2"
                },
                "tags": {
                    "rankingType": "UNRANKED",
                    "serverType": "QUICK2V2"
                },
                "titleId": "stunlock-studios-battlerite"
            },
            "relationships": {
                "assets": {
                    "data": [
                        {
                            "type": "asset",
                            "id": "626990a2-3541-11e8-b891-0a586460a616"
                        }
                    ]
                }

In [22]:
print(json.dumps(data['data'][0], indent=4))

{
    "type": "match",
    "id": "34F63A9E2F544B53913C0AD840572330",
    "attributes": {
        "createdAt": "2018-04-01T00:00:00Z",
        "duration": 303,
        "gameMode": "1733162751",
        "patchVersion": "44504",
        "shardId": "global",
        "stats": {
            "mapID": "319DDC57E70174B6C85EF137BAF34E9E",
            "type": "QUICK2V2"
        },
        "tags": {
            "rankingType": "UNRANKED",
            "serverType": "QUICK2V2"
        },
        "titleId": "stunlock-studios-battlerite"
    },
    "relationships": {
        "assets": {
            "data": [
                {
                    "type": "asset",
                    "id": "626990a2-3541-11e8-b891-0a586460a616"
                }
            ]
        },
        "rosters": {
            "data": [
                {
                    "type": "roster",
                    "id": "4b490036-c870-4293-8e6f-06eb60c5ca30"
                },
                {
                    "type": "roster",

In [6]:
print(json.dumps(data['included'], indent=4))

[
    {
        "type": "asset",
        "id": "626990a2-3541-11e8-b891-0a586460a616",
        "attributes": {
            "URL": "https://cdn.gamelockerapp.com/stunlock-studios-battlerite/global/2018/04/01/00/12/626990a2-3541-11e8-b891-0a586460a616-telemetry.json",
            "createdAt": "2018-04-01T00:12:37Z",
            "description": "",
            "name": "telemetry"
        }
    },
    {
        "type": "roster",
        "id": "16e57781-b113-450e-997a-b69d01e745cb",
        "attributes": {
            "shardId": "global",
            "stats": {
                "division": -1,
                "divisionRating": -1,
                "league": -1,
                "losses": 1,
                "matchRegion": "eu",
                "placementGamesLeft": 7,
                "prevDivision": -1,
                "prevDivisionRating": -1,
                "prevLeague": -1,
                "prevLosses": 0,
                "prevPlacementGamesLeft": 8,
                "prevWins": 2,
          

In [7]:
def convert_to_df(data):
    '''
    Get match data, and store in a DataFrame
    '''
    
    # Get telemetry urls
    telemetry_urls = {}
    for entry in data['included']:
        if entry['type'] == 'asset':
            telemetry_urls[str(entry['id'])] = entry['attributes']['URL']

    # Get match data
    match_data = {'date':[], 'match_id':[], 'game_mode':[], 'ranked':[], 'character':[], 'league':[], 'win':[]}
    for entry in data['data']:
        if entry['attributes']['stats']['type'] in ['QUICK2V2', 'QUICK3V3', 'LEAGUE2V2', 'LEAGUE3V3']:
            # Get date
            date = datetime.strptime(entry['attributes']['createdAt'], '%Y-%m-%dT%H:%M:%SZ')
            match_data['date'] = date
            
            # Get match id
            match_id = entry['id']

            # Get game mode (2v2 or 3v3)
            game_mode = entry['attributes']['stats']['type']

            # Get telemetry data
            asset_id = entry['relationships']['assets']['data'][0]['id']
            url = telemetry_urls[asset_id]
            r = requests.get(url, headers={"Accept":"application/vnd.api+json"})
            telemetry = r.json()

            # Get player info
            players = []
            for entry in telemetry:
                # Determine winning team
                if entry['type'] == 'Structures.MatchFinishedEvent':
                    if entry['dataObject']['teamOneScore'] > entry['dataObject']['teamTwoScore']:
                        winner = 1
                    else:
                        winner = 2
                
                # Gather player info
                if entry['type'] == 'Structures.MatchReservedUser':
                    players.append(entry)

            for player in players:
                # Gather player info
                character = player['dataObject']['character']
                league = player['dataObject']['league']
                if player['dataObject']['team'] == winner:
                    win = True
                else:
                    win = False

                # Store player info
                match_data['character'].append(character)
                match_data['league'].append(league)
                match_data['win'].append(win)
            
            # Determine if the game was ranked
            if players[0]['dataObject']['rankingType'] == 'RANKED':
                ranked = True
            else:
                ranked = False

            count = len(players)
            # Store remaining data
            match_data['match_id'] += [match_id]*count
            match_data['game_mode'] += [game_mode]*count
            match_data['ranked'] += [ranked]*count
            
    # Convert into DataFrame
    DataFrame = pd.DataFrame(match_data)
    
    return DataFrame

In [8]:
def get_chunk(df=pd.DataFrame(), year=2018, month=1, day=1):
    '''
    Get chunk of data
    '''
    
    # Initialize start and end dates
    start = datetime(year, month, day, 0, 0, 0, 0)
    end = start + timedelta(days=1)
    
    # Get data from first page
    url = "https://api.dc01.gamelockerapp.com/shards/global/matches"
    data = get_data(url, start, end)
    temp = convert_to_df(data)
    df = pd.concat([df, temp], ignore_index=True)
    
    # Get data from next pages
    for i in range(9):
        # Get the next page
        url = data['links']['next']
        data = get_data(url)
        
        # Create DataFrame
        temp = convert_to_df(data)

        # Concatenate DataFrames
        df = pd.concat([df, temp], ignore_index=True)
        
    # Delay requests
    time.sleep(50)
    
    return df

In [9]:
# Initialize empty DataFrame
match_df = pd.DataFrame()

# Cycle through Jan-March
for month in range(1,4):
    # Cycle through various days of the month
    for day in range(1,29,3):
        # Update DataFrame
        match_df = get_chunk(match_df, 2018, month, day)
        # Save DataFrame
        match_df.to_csv('match_data', index=False)

# Cycle through first week of April
for day in range(1,8):
    # Update DataFrame
    match_df = get_chunk(match_df, 2018, 4, day)
    # Save DataFrame
    match_df.to_csv('match_data', index=False)

# Rename entries of 'game_mode' column to either 2v2, or 3v3
match_df['game_mode'] = match_df['game_mode'].apply(lambda x: x[-3:])

# Save DataFrame
match_df.to_csv('compiled_data\match_data', index=False)